In [2]:
"""
Created Using Python 3.6
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

In [6]:
data = loadmat("data/expdata07.mat")

In [9]:
def load_soliton_data(filename):
    data = loadmat(filename)
    A0 = data["A0"]
    A0_error = data["A0_error"]
    U0 = data["U0"]
    A = data["Amat"]
    t = data["t_vec"]
    z = data["z_vec"]
    return t, z, A, U0, A0, A0_error